# Obtaining Our Data - Lab

## Introduction
In this lab you'll practice your munging and transforming skills in order to load in your data to solve a regression problem.

## Objectives
You will be able to:
* Understand the ETL process and the steps it consists of
* Understand the challenges of working with data from multiple sources 

## Task Description

Your boss gives you a general description of some of the datasets at your disposal for analyzing weekly store sales. They're eventually looking for you to build a model to help determine what factors impact sales, and model future sales forecasting for business planning.  
  
Most of the properietary store data sits in the company sql database, accessible by all managers and above. The database is called **Walmart.db** Your boss provides you with the following basic schema:  

<img src='db_schema.jpg' width=500>  

She then tells you that she's put together a second dataset on general economy statistics for the various dates that she would also like you to incorporate in your analysis. That data, she says, is stored in a file **economy_data.csv**.

As a first step in creating your model for providing recommendations and projections, load and synthesize these disperate datasets into a singular unified DataFrame. Then save your results to a file **Merged_Store_Data.csv**.

Make sure you check the various data types and merge appropriately.

# Import Walmart.db Database using SQL

Store Walmart.db within a dataframe

In [1]:
# Your code here
import sqlite3
import pandas as pd

#connection
con = sqlite3.connect('Walmart.db')
#cursor
cur = con.cursor()
#data
cur.execute('''SELECT * from Sales join store_details using(Store);''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]


### Explore Walmart.db dataframe

In [2]:
print(df.shape)
df.head()

(452192, 7)


,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size
0,1,1,2010-02-05,24924.50,False,A,151315
1,1,1,2010-02-12,46039.49,True,A,151315
2,1,1,2010-02-19,41595.55,False,A,151315
3,1,1,2010-02-26,19403.54,False,A,151315
4,1,1,2010-03-05,21827.90,False,A,151315


In [3]:
print('There are ' + str(df.shape[0]) + ' samples and 7 features on this database')

There are 452192 samples and 7 features on this database


In [4]:
print('There are ' + str(len(df.Date.unique())) + ' unique dates.')
print('There are ' + str(len(df.Store.unique())) + ' unique stores.')

There are 143 unique dates.
There are 45 unique stores.


In [5]:
print('Earliest sample date: ' + str(df.Date.min()))
print('Latest sample date: ' + str(df.Date.max()))

Earliest sample date: 2010-02-05
Latest sample date: 2012-10-26


# Import Economy CSV file as Dataframe

In [6]:
df_econ = pd.read_csv('economy_data.csv')

### Explore Economy dataframe

In [7]:
df_econ.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [8]:
print('There are ' + str(df_econ.shape[0]) + ' samples on this dataset.')

There are 8190 samples on this dataset.


In [9]:
print('There are ' + str(len(df_econ.Date.unique())) + ' unique dates.')
print('There are ' + str(len(df_econ.Store.unique())) + ' unique stores.')

There are 182 unique dates.
There are 45 unique stores.


In [10]:
print('Earliest sample date: ' + str(df_econ.Date.min()))
print('Latest sample date: ' + str(df_econ.Date.max()))

Earliest sample date: 2010-02-05
Latest sample date: 2013-07-26


## Finding 1: Sales Data stops on 2012_10_26

Therefore, we have no use for samples beyond that point in the df_econ dataframe.

In [11]:
#convert dates to pandas datetime
df_econ['Date'] = pd.to_datetime(df_econ['Date'])

In [12]:
# delete entries from df_econ beyond 2012_10_26
import datetime
# drop row if Date > 2012_10_26
date = pd.to_datetime('2012-10-26')
df_econ = df_econ[df_econ['Date'] <= date]
df_econ.tail()


,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
8146,45,2012-09-28,64.88,3.997,4556.61,20.64,1.50,1601.01,3288.25,192.013558,8.684,False
8147,45,2012-10-05,64.89,3.985,5046.74,NaN,18.82,2253.43,2340.01,192.170412,8.667,False
8148,45,2012-10-12,54.47,4.000,1956.28,NaN,7.89,599.32,3990.54,192.327265,8.667,False
8149,45,2012-10-19,56.47,3.969,2004.02,NaN,3.18,437.73,1537.49,192.330854,8.667,False
8150,45,2012-10-26,58.85,3.882,4018.91,58.08,100.00,211.94,858.33,192.308899,8.667,False


In [13]:
df_econ.shape

(6435, 12)

In [14]:
# Let's check how many unique dates are now in the df_econ dataframe
len(df_econ['Date'].unique())

143

Success! Now we have 143 unique dates that hopefully coincide with the dates in the sales dataframe. Let's check that.

In [15]:
# first convert dates to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [16]:
# check for NaN values in various columns
na = []
for column in df.columns:
    na.append(sum(df[column].isna()))
na


[0, 0, 0, 0, 0, 0, 0]

In [17]:
# No NaN values in df

In [18]:
na = []
for column in df_econ.columns:
    na.append(sum(df_econ[column].isna()))
na

[0, 0, 0, 0, 4155, 4798, 4389, 4470, 4140, 0, 0, 0]

In [19]:
df_econ.shape

(6435, 12)

Although these NANs in Markdowns are a great portion, their presence may
be valuable (lack of a markdown may be related with less sales, for instance)

In [20]:
df.shape, df_econ.shape

((452192, 7), (6435, 12))

In [21]:
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size
0,1,1,2010-02-05,24924.50,False,A,151315
1,1,1,2010-02-12,46039.49,True,A,151315
2,1,1,2010-02-19,41595.55,False,A,151315
3,1,1,2010-02-26,19403.54,False,A,151315
4,1,1,2010-03-05,21827.90,False,A,151315


In [29]:
df_econ.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,True
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,True
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,True
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,True


In [30]:
df.IsHoliday = df.IsHoliday.astype(bool)
df_econ.IsHoliday = df.IsHoliday.astype(bool)

In [38]:
merged = df.merge(df_econ)

merged


,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2010-02-05,24924.50,True,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
1,1,2,2010-02-05,50605.27,True,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
2,1,3,2010-02-05,13740.12,True,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
3,1,4,2010-02-05,39954.04,True,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
4,1,5,2010-02-05,32229.38,True,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
5,1,6,2010-02-05,5749.03,True,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
6,1,7,2010-02-05,21084.08,True,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
7,1,8,2010-02-05,40129.01,True,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
8,1,9,2010-02-05,16930.99,True,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
9,1,10,2010-02-05,30721.50,True,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106


In [37]:
merged.shape

(452192, 16)

In [39]:
merged.to_csv('Merged_Store_Data.csv',index=False)

## Summary
Nice work! You're working more and more independently through the workflow, and ensuring data integrity!